In [10]:
import glob
from sam.stress_addition_model import (
    sam_prediction,
    Predicted_LCs,
    get_sam_lcs,
    SAM_Setting,
    OLD_STANDARD,
    Transforms
)
from sam.helpers import compute_lc, compute_lc_from_curve
from sam.plotting import plot_sam_prediction
from sam.data_formats import ExperimentData, read_data, load_datapoints
import os
import matplotlib.pyplot as plt
import pandas as pd
import argparse
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [4]:
SETTINGS = OLD_STANDARD


rows = []

for path, data, name, val in tqdm(load_datapoints()):

    main_fit, stress_fit, sam_sur, sam_stress, additional_stress = sam_prediction(
        data.main_series,
        val,
        data.meta,
        settings=SETTINGS,
    )

    lcs = get_sam_lcs(stress_fit=stress_fit, sam_sur=sam_sur, meta=data.meta)

    row = {
        "path": path,
        "stressor": name,
        "stress_lc10": lcs.stress_lc10,
        "stress_lc50": lcs.stress_lc50,
        "sam_lc10": lcs.sam_lc10,
        "sam_lc50": lcs.sam_lc50,
        "survival_max": data.meta.max_survival,
        "lc1":main_fit.lc1,
        "lc99":main_fit.lc99,
    }

    rows.append(row)

df = pd.DataFrame(rows)


100%|██████████| 47/47 [00:08<00:00,  5.58it/s]


In [6]:
df.isna().sum()

path            0
stressor        0
stress_lc10     0
stress_lc50     0
sam_lc10        0
sam_lc50        0
survival_max    0
lc1             0
lc99            0
dtype: int64

In [7]:
df.lc1 = df.lc1.fillna(0)
normed = df.copy()
cols = ["stress_lc10", "stress_lc50", "sam_lc10", "sam_lc50"]
normed[cols] -= df.lc1.values[:,None]
normed[cols] = np.maximum(0,normed[cols])
normed[cols] /= (df.lc99.values[:,None] - df.lc1.values[:,None])
normed[cols].describe()

,stress_lc10,stress_lc50,sam_lc10,sam_lc50
count,47.000000,47.000000,47.000000,47.000000
mean,0.006685,0.045231,0.001593,0.034377
std,0.015372,0.059702,0.004857,0.047418
min,0.000000,0.000000,0.000000,0.000196
25%,0.000000,0.006061,0.000000,0.005950
50%,0.001336,0.026069,0.000000,0.016040
75%,0.004619,0.055585,0.000293,0.042344
max,0.087793,0.324615,0.022100,0.245987


In [11]:
fifty = normed.dropna(subset=["stress_lc50"])
mean_squared_error(fifty.stress_lc50, fifty.sam_lc50), mean_absolute_error(fifty.stress_lc50, fifty.sam_lc50), r2_score(fifty.stress_lc50, fifty.sam_lc50)

(0.0007156456957927874, 0.016424423601822676, 0.7948579418253063)

In [12]:
ten = normed.dropna(subset=["stress_lc10"])
mean_squared_error(ten.stress_lc10, ten.sam_lc10), mean_absolute_error(ten.stress_lc10, ten.sam_lc10), r2_score(ten.stress_lc10, ten.sam_lc10)

(0.0002398438232910721, 0.0058455356155869605, -0.037056949763056624)